1. nan들을 fillna로 해결하는 코드가 있는지 (effective한지 체크)<br>
2. MACD, RSI, STD< SKEW 등의 features 넣어주긔<br>
3. 모델 ensembling / stacking / NN 테스트

In [1]:
import os
import gc
import sys
import joblib
import numpy as np
%load_ext cudf.pandas      # pandas speed up
import pandas as pd 
# pd.set_option("display.max_columns", 50)
# pd.set_option("display.max_rows", 50)

from warnings import filterwarnings
filterwarnings(action="ignore", category=pd.errors.PerformanceWarning)
# pd.options.mode.chained_assignment = None # pd 경고문을 안 뜨게 해

# import seaborn as sns
# import matplotlib.pyplot as plt
# %matplotlib inline

import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, TimeSeriesSplit

import json
from itertools import combinations

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        
PATH = "/kaggle/input/optiver-trading-at-the-close/"

# median_vol = pd.read_csv("/kaggle/input/optiver-memoryreduceddatasets/MedianVolV2.csv")
# median_vol.index.name = "stock_id"
# median_vol = median_vol[["overall_medvol","first5min_medvol","last5min_medvol"]]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
change_dtypes = {"stock_id": np.uint8,
                "date_id": np.uint16, 
                "seconds_in_bucket": np.uint16,
                "imbalance_buy_sell_flag": np.int8,
                "time_id": np.uint16}

# train_df = pd.read_csv(PATH + "train.csv", dtype=change_dtypes).drop(columns=["row_id","time_id"], axis=1)
train_df2 = pd.read_parquet("/kaggle/input/optiver-datapreparation/XTrainIntCmp.parquet")
y = pd.read_parquet("/kaggle/input/optiver-datapreparation/Ytrain.parquet")
# test_df = pd.read_csv(PATH + "example_test_files/test.csv", dtype=change_dtypes).drop(columns=["row_id","time_id"], axis=1)

# train_df2.dropna(subset=["target"], inplace=True)
# train_df2.reset_index(drop=True, inplace=True)

del change_dtypes

In [3]:
train_df2["far_price"] = train_df2["far_price"][train_df2["far_price"].isna() & train_df2["date_id"] < 300].fillna(0)

In [4]:
train_df2["near_price"] = train_df2["near_price"][train_df2["near_price"].isna() & train_df2["date_id"] < 300].fillna(0)

In [6]:
def reduce_mem_usage(df, verbose=0):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print(f"starting memory: {start_mem}")
    for col in df.columns:
        col_type = df[col].dtype
        
        # check if the column's data type is not "object" (i.e. numeric)
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # check if the column's data type is an integer
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int16)
                    
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                    
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                    
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
                    
                else:
                    # check if the column's data type is a float
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float32)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float32)
                        
        if verbose:
            print(f"start memory: {start_mem}")
            end_mem = df.memory_usage().sum() / 1024 ** 2
            print(f"current memory: {end_mem}")
        
        return df

In [7]:
from numba import njit, prange

@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))
    for i in prange(num_combinations):
        a,b,c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val

            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)
            
    return imbalance_features


def calculate_triplet_imbalance_numba(price, df):
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a,b,c in combinations(price, 3)]
    features_array = compute_triplet_imbalance(df_values, comb_indices)
    columns = [f"{a}_{b}_{c}_imb2" for a,b,c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)
    return features

In [8]:
# https://www.kaggle.com/code/chinzorigtganbat/feature-generation-technical-analysis-functions
@njit(fastmath=True)
def rolling_average(arr, window):
    n = len(arr)
    result = np.empty(n)
    result[:window] = np.nan
    cumsum = np.cumsum(arr)
    
    for i in range(window, n):
        result[i] = (cumsum[i] - cumsum[i-window]) / window
    return result


@njit(parallel=True)
def compute_rolling_averages(df_values, window_sizes):
    num_rows, num_features = df_values.shape
    num_windows = len(window_sizes)
    rolling_features = np.empty((num_rows, num_features, num_windows))
    
    for feature_idx in prange(num_features):
        for window_idx, window in enumerate(window_sizes):
            rolling_features[:, feature_idx, window_idx] = rolling_average(df_values[:, feature_idx], window)
    return rolling_features


@njit(parallel=True)
def calculate_rsi(prices, period=14):
    rsi_values = np.zeros_like(prices)
    
    for col in prange(prices.shape[1]):
        price_data = prices[:, col]
        delta = np.zeros_like(price_data)
        delta[1:] = price_data[1:] - price_data[:-1]
        gain = np.where(delta > 0, delta, 0)
        loss = np.where(delta < 0, -delta, 0)
        
        avg_gain = np.mean(gain[:period])
        avg_loss = np.mean(loss[:period])
        
        if avg_loss != 0:
            rs = avg_gain / avg_loss
        else:
            rs = 1e-9
            
        rsi_values[:period, col] = 100 - (100 / (1+rs))
        
        for i in prange(period-1, len(price_data)-1):
            avg_gain = (avg_gain * (period - 1) + gain[i]) / period
            avg_loss = (avg_loss * (period - 1) + loss[i]) / period
            
            if avg_loss != 0:
                rs = avg_gain / avg_loss
            else:
                rs = 1e-9
            
            rsi_values[i+1, col] = 100 - (100 / (1 + rs))
    return rsi_values


@njit(parallel=True)
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    rows, cols = data.shape
    macd_values = np.empty((rows, cols))
    signal_line_values = np.empty((rows, cols))
    histogram_values = np.empty((rows, cols))
    
    for i in prange(cols):
        short_ema = np.zeros(rows)
        long_ema = np.zeros(rows)
        for j in range(1, rows):
            short_ema[j] = (data[j, i] - short_ema[j - 1]) * (2 / (short_window + 1)) + short_ema[j - 1]
            long_ema[j] = (data[j, i] - long_ema[j - 1]) * (2 / (long_window + 1)) + long_ema[j - 1]
            
        macd_values[:, i] = short_ema - long_ema
        signal_line = np.zeros(rows)
        
        for j in range(1, rows):
            signal_line[j] = (macd_values[j,i] - signal_line[j - 1]) * (2 / (signal_window + 1)) + signal_line[j - 1]
            
        signal_line_values[:, i] = signal_line
        histogram_values[:, i] = macd_values[:, i] - signal_line
    return macd_values, signal_line_values, histogram_values



@njit(parallel=True)
def calculate_bband(data, window=20, num_std_dev=2):
    num_rows, num_cols = data.shape
    upper_bands = np.zeros_like(data)
    lower_bands = np.zeros_like(data)
    mid_bands = np.zeros_like(data)
    
    for col in prange(num_cols):
        for i in prange(window-1, num_rows):
            window_slice = data[i-window+1:i+1, col]
            mid_bands[i, col] = np.mean(window_slice)
            std_dev = np.std(window_slice)
            upper_bands[i, col] = mid_bands[i, col] + num_std_dev * std_dev
            lower_bands[i, col] = mid_bands[i, col] - num_std_dev * std_dev
            
    return upper_bands, mid_bands, lower_bands

In [9]:
def generate_ta(df):
    # define lists of price and size-related column names
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    
    for stock_id, values in df.groupby(["stock_id"])[prices]:
        # RSI: relative strength index
        col_rsi = [f"rsi_{col}" for col in values.columns]
        rsi_values = calculate_rsi(values.values)
        df.loc[values.index, col_rsi] = rsi_values
        gc.collect()
        
        # MACD
        macd_values, signal_line_values, histogram_values = calculate_macd(values.values)
        col_macd = [f"macd_{col}" for col in values.columns]
        col_signal = [f"macd_sig_{col}" for col in values.columns]
        col_hist = [f"macd_hist_{col}" for col in values.columns]
        
        df.loc[values.index, col_macd] = macd_values
        df.loc[values.index, col_signal] = signal_line_values
        df.loc[values.index, col_hist] = histogram_values
        gc.collect()
        
        # bollinger Bands
        bband_upper_values, bband_mid_values, bband_lower_values = calculate_bband(values.values, window=20, num_std_dev=2)
        col_bband_upper = [f"bband_upper_{col}" for col in values.columns]
        col_bband_mid = [f"bband_mid_{col}" for col in values.columns]
        col_bband_lower = [f"bband_lower_{col}" for col in values.columns]
        
        df.loc[values.index, col_bband_upper] = bband_upper_values
        df.loc[values.index, col_bband_mid] = bband_mid_values
        df.loc[values.index, col_bband_lower] = bband_lower_values
        gc.collect()
        
    return df, [col for col in df.columns if col not in ["date_id","stock_id"]]

In [10]:
def create_features(df, reduce_memory=True):
#     cols_to_drop = ["imbalance_buy_sell_flag"]
    prices = ["reference_price","far_price","near_price","ask_price","bid_price","wap"]
    sizes = ["matched_size","bid_size","ask_size","imbalance_size"]
    
    df["volume"] = df.eval("ask_size + bid_size")
#     df["mid_price"] = df.eval("(ask_price + bid_size) / 2")
    df["liquidity_imbalance"] = df.eval("(bid_size - ask_size)/(bid_size + ask_size)")
#     df["matched_imbalance"] = df.eval("(imbalance_size - matched_size)/(matched_size + imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")
    
    df["price_spread"] = df.eval("ask_price - bid_price")
    df["imbalance_ratio"] = df.eval("imbalance_size / matched_size")
    
    df["ask_volume"] = df.eval("ask_size * ask_price")
    df["bid_volume"] = df.eval("bid_size * bid_price")
    
    df["ask_bid_volumes_diff"] = df["ask_volume"] - df["bid_volume"]
    
    df["bid_size_over_ask_size"] = df["bid_size"].div(df["ask_size"]) # size imbalance
    df["bid_price_over_ask_price"] = df["bid_price"].div(df["ask_price"])
    
    df["imbalance_buy_flag"] = np.where(df["imbalance_buy_sell_flag"]==1, True, False)
    df["imbalance_sell_flag"] = np.where(df["imbalance_buy_sell_flag"]==-1, True, False)
    
    # create features for pairwise price imbalances
    # https://www.kaggle.com/code/judith007/lb-5-3393-rapids-gpu-speeds-up-feature-engineer
    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")
    
    # V2 features # 아래 피쳐들 의미를 잘 모르겠땅
    df["imbalance_momentum"] = df.groupby(["stock_id"])["imbalance_size"].diff(periods=1)/df["matched_size"]    
    df["spread_intensity"] = df.groupby(["stock_id"])["price_spread"].diff()
    df["price_pressure"] = df["imbalance_size"] * (df["ask_price"] - df["bid_price"])
    df["market_urgency"] = df["price_spread"] * df["liquidity_imbalance"]
    df["depth_pressure"] = (df["ask_size"] - df["bid_size"]) * (df["far_price"] - df["near_price"])
    
    # V3 features
    for col in ["matched_size","imbalance_size","reference_price","imbalance_buy_sell_flag"]:
        for window in [1,2,3,10]:      # 이거 숫자들의 의미가 뭐지?
            df[f"{col}_shift_{window}"] = df.groupby("stock_id")[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby("stock_id")[col].pct_change(window)
            
    # calculate diff features for specific columns
    for col in ["ask_price","bid_price","ask_size","bid_size"]:
        for window in [1,2,3,10]:
            df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)
            
    for c in [["ask_price","bid_price","wap","reference_price"], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
    
    if reduce_memory:
        print(f"Reducing memory usage... (Current: {df.memory_usage().sum() / 1024 / 1024:.1f} MB)")
        df = df.astype({"stock_id":"uint8",
                       "seconds_in_bucket":"uint16",
                       # converts all float64s to float32s
                       **{col:"float32" for col in df.columns if df[col].dtype == "float64"}})
        print(f"Memory usage after reduction: {df.memory_usage().sum() / 1024 / 1024:.1f} MB")
    
    return df

In [11]:
train_df2 = create_features(train_df2)
train_df2, ta_cols = generate_ta(train_df2)
ta_cols = [x for x in ta_cols if (x != "target")]
train_df2 = reduce_mem_usage(train_df2)
date_ids = train_df2["date_id"].values

train_df2.drop(columns=["date_id","stock_id"
                       # ,"target"
                       ], inplace=True)

# feat_importances = ["seconds_in_bucket","market_urgency","near_price_ask_price_imb",\
# "reference_price_wap_imb","wap","far_price_bid_price_imb","reference_price_bid_price_imb",\
# "ask_price_diff_2","imbalance_size_shift_1","far_price_near_price_imb",\
# "matched_size_ask_size_imbalance_size_imb2",\
# "matched_size_ret_10","price_spread","matched_size_shift_10","bid_price_diff_1","spread_intensity",\
# "ask_price_bid_price_reference_price_imb2","reference_price_ask_price_imb",\
# "matched_size_bid_size_imbalance_size_imb2",
# "reference_price_ret_3","far_price_ask_price_imb",\
#                     "imbalance_momentum","bid_price","ask_price",\
# "ask_price_diff_1","imbalance_ratio","imbalance_buy_sell_flag",\
#                     "reference_price_ret_10",\
                    
# "reference_price","volume","bid_size_ask_size_imbalance_size_imb2","imbalance_size",\
# "bid_volume","matched_size_shift_1","matched_size_bid_size_ask_size_imb2",\
                    
# "bid_price_over_ask_price","bid_price_wap_imb","reference_price_shift_3",\
# "reference_price_ret_1","bid_size_diff_1","reference_price_shift_1",\
#                     "ask_size","bid_price_diff_10",\
# "matched_size_ret_1","matched_size_shift_2","matched_size_shift_3",\
#                     "ask_price_diff_10",\
# "ask_size_diff_1","ask_volume","ask_price_bid_price_wap_imb2",\
#                     "reference_price_ret_2","reference_price_shift_2",\
# "matched_size_ret_3","bid_size_diff_3","matched_size_ret_2",\
#                     "price_pressure","bid_size_diff_2",\
# "ask_size_diff_3","reference_price_far_price_imb",\
#                     "liquidity_imbalance","bid_price_wap_reference_price_imb2",\
# "bid_size_diff_10","ask_size_diff_10",\
#                 "bid_size_over_ask_size","ask_bid_volumes_diff",\
# "depth_pressure"
#                    ] + ta_cols #, "overall_medvol","target"


train_df2 = train_df2[ta_cols]
# del feat_importances
gc.collect()

# import seaborn as sns
# sns.heatmap(np.around(train_df.corr(), 2)) #, annot=True) 

/opt/conda/lib/python3.10/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'comb_indices' of function 'compute_triplet_imbalance'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../tmp/ipykernel_33/3417327308.py", line 3:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/opt/conda/lib/python3.10/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'comb__indices' of function '__numba_parfor_gufunc_0x7e6d340ab340'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<string>", lin

Reducing memory usage... (Current: 3926.3 MB)
Memory usage after reduction: 2008.1 MB
starting memory: 3815.4894256591797


0

In [12]:
# # IR: Information Ratio     as a feature..넣어주긔 
# def calculate_information_ratio(x1, x2):
#     assert len(x1) == len(x2)
#     interval = 495000  # 45 days

In [13]:
lgb_params2 = {'objective': 'mae',
 'n_estimators': 5500,
 'num_leaves': 128,
 'subsample': 0.6,
 'colsample_bytree': 0.8,
 'learning_rate': 5e-05,
 'max_depth': 11,
 'n_jobs': 4,
 'device': 'gpu',
 'verbosity': -1,
 'importance_type': 'gain'}

# lgb_params3 = {
#     "objective": "mae",
#     "n_estimators": 6000,
#     "num_leaves": 256,
#     "subsample": 0.6,
#     'learning_rate': 0.00871, 
#     'max_depth': 11,
#     "colsample_bytree" : 0.8,
#     "n_jobs": 4,
#     "device": "gpu",
#     "verbosity": -1,
#     "importance_type": "gain",
#     "random_state": 42,
#     "max_bin": 247
# }

gc.collect()

0

In [14]:
# https://www.kaggle.com/code/sunghoshim/optiver-checking-feature-importance
# model = lgb.Booster(model_file='/kaggle/input/explained-singel-model-optiver/modelitos_para_despues/doblez-conjunto.txt')
# importances = model.feature_importance(importance_type="gain")
# feature_names = model.feature_name()

# ser = pd.Series(importances, index=feature_names)
# del importances, feature_names
# ser_sorted = ser.sort_values().astype(int)

# import matplotlib.pyplot as plt
# max_num_features = 177
# fig, ax = plt.subplots(figsize=(12, 15))
# bars = ax.barh(ser_sorted.index[-max_num_features:],
#               ser_sorted[-max_num_features:])
# ax.bar_label(bars, padding=5, fmt="{:,.0f}")
# del model
# gc.collect()

In [15]:
# important_features = ser_sorted.index[-80:]
# important_features = list(important_features)

In [16]:
feats = ['ask_bid_volumes_diff', 'ask_price', 'ask_price_bid_price_imb', 'ask_price_bid_price_reference_price_imb2', 
         'ask_price_bid_price_wap_imb2', 'ask_price_diff_1', 'ask_price_diff_10', 'ask_price_diff_2', 'ask_price_diff_3', 
         'ask_price_wap_imb', 'ask_price_wap_reference_price_imb2', 'ask_size', 'ask_size_diff_1', 'ask_size_diff_10',
         'ask_size_diff_2', 'ask_size_diff_3', 'ask_volume', 'bband_lower_ask_price', 'bband_lower_bid_price', 'bband_lower_far_price', 
         'bband_lower_near_price', 'bband_lower_reference_price', 'bband_lower_wap', 'bband_mid_ask_price', 'bband_mid_bid_price', 
         'bband_mid_far_price', 'bband_mid_near_price', 'bband_mid_reference_price', 'bband_mid_wap', 'bband_upper_ask_price', 'bband_upper_bid_price', 'bband_upper_far_price', 'bband_upper_near_price', 'bband_upper_reference_price', 'bband_upper_wap', 'bid_price', 'bid_price_diff_1', 'bid_price_diff_10', 'bid_price_diff_2', 'bid_price_diff_3', 'bid_price_over_ask_price', 'bid_price_wap_imb', 'bid_price_wap_reference_price_imb2', 'bid_size', 'bid_size_ask_size_imbalance_size_imb2', 'bid_size_diff_1', 'bid_size_diff_10', 'bid_size_diff_2', 'bid_size_diff_3', 'bid_size_over_ask_size', 'bid_volume', 'depth_pressure', 'far_price', 'far_price_ask_price_imb', 'far_price_bid_price_imb', 'far_price_near_price_imb', 'far_price_wap_imb', 'imbalance_buy_flag', 'imbalance_buy_sell_flag', 'imbalance_buy_sell_flag_ret_1', 'imbalance_buy_sell_flag_ret_10', 'imbalance_buy_sell_flag_ret_2', 'imbalance_buy_sell_flag_ret_3', 'imbalance_buy_sell_flag_shift_1', 'imbalance_buy_sell_flag_shift_10', 'imbalance_buy_sell_flag_shift_2', 'imbalance_buy_sell_flag_shift_3', 'imbalance_momentum', 'imbalance_ratio', 'imbalance_sell_flag', 'imbalance_size', 'imbalance_size_ret_1', 'imbalance_size_ret_10', 'imbalance_size_ret_2', 'imbalance_size_ret_3', 'imbalance_size_shift_1', 'imbalance_size_shift_10', 'imbalance_size_shift_2', 'imbalance_size_shift_3', 'liquidity_imbalance', 'macd_ask_price', 'macd_bid_price', 'macd_far_price', 'macd_hist_ask_price', 'macd_hist_bid_price', 'macd_hist_far_price', 'macd_hist_near_price', 'macd_hist_reference_price', 'macd_hist_wap', 'macd_near_price', 'macd_reference_price', 'macd_sig_ask_price', 'macd_sig_bid_price', 'macd_sig_far_price', 'macd_sig_near_price', 'macd_sig_reference_price', 'macd_sig_wap', 'macd_wap', 'market_urgency', 'matched_size', 'matched_size_ask_size_imbalance_size_imb2', 'matched_size_bid_size_ask_size_imb2', 'matched_size_bid_size_imbalance_size_imb2', 'matched_size_ret_1', 'matched_size_ret_10', 'matched_size_ret_2', 'matched_size_ret_3', 'matched_size_shift_1', 'matched_size_shift_10', 'matched_size_shift_2', 'matched_size_shift_3', 'near_price', 'near_price_ask_price_imb', 'near_price_bid_price_imb', 'near_price_wap_imb', 'price_pressure', 'price_spread', 'reference_price', 'reference_price_ask_price_imb', 'reference_price_bid_price_imb', 'reference_price_far_price_imb', 'reference_price_near_price_imb', 'reference_price_ret_1', 'reference_price_ret_10', 'reference_price_ret_2', 'reference_price_ret_3', 'reference_price_shift_1', 'reference_price_shift_10', 'reference_price_shift_2', 'reference_price_shift_3', 'reference_price_wap_imb', 'rsi_ask_price', 'rsi_bid_price', 'rsi_far_price', 'rsi_near_price', 'rsi_reference_price', 'rsi_wap', 'seconds_in_bucket', 'size_imbalance', 'spread_intensity', 'volume', 'wap']
feats = [f for f in feats if (f in train_df2.columns)]
feats = list(np.unique(feats))

In [17]:
train_df2 = train_df2.loc[:, ~train_df2.columns.duplicated()].copy()
feature_name = feats
# del important_features, ser_sorted, 
del ta_cols, feats
# train_df.dropna(inplace=True)
# train_df = train_df.reset_index()
gc.collect()

# print(train_df2.shape)
# print(len(y))

0

In [ ]:
import xgboost as xgb
import catboost as cbt

n_fold = 5
index = np.arange(len(train_df2))
model_dict = {"lgb": lgb.LGBMRegressor(objective="regression_l1",
                                     n_estimators=500),
              "xgb": xgb.XGBRegressor(tree_method="hist",
                                     objective="reg:absoluteerror",
                                     n_estimator=500),
              "cbt": cbt.CatBoostRegressor(objective="MAE",
                                          iterations=3000)
             }
is_training = True
def train(model_dict, modelname="lgb"):
    if is_training:
        model = model_dict[modelname]
        model.fit(train_df2[index%n_fold != i], y[index%n_fold != i],
                 eval_set=[(train_df2[index%n_fold == i], y[index%n_fold == i])],
                 verbose=10, early_stopping_rounds=100)
        model.append(model)
        joblib.dump(model, f"/kaggle/working/{modelname}.model")
        
    
for i in range(n_fold):
    train(model_dict, "lgb")

/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


In [ ]:
# models = []
# scores = []

# num_folds = 10
# fold_size = 480 // num_folds

# for i in range(num_folds):
#     start = i * fold_size
#     end = start + fold_size
    
#     # Define the purged set ranges
#     purged_before_start = start - 2
#     purged_before_end = start + 2
#     purged_after_start = end - 2
#     purged_after_end = end + 2
    
#     # Exclude the purged ranges from the test set
#     purged_set = ((date_ids >= purged_before_start) & (date_ids <= purged_before_end)) | \
#                  ((date_ids >= purged_after_start) & (date_ids <= purged_after_end))
    
#     # Define test_indices excluding the purged set
#     test_indices = (date_ids >= start) & (date_ids < end) & ~purged_set
#     train_indices = ~test_indices & ~purged_set
    
#     df_fold_train = train_df2[train_indices]
#     df_fold_train_target = y[train_indices]
#     df_fold_valid = train_df2[test_indices]
#     df_fold_valid_target = y[test_indices]

#     print(f"Fold {i+1} Model Training")
    
#     # Train a LightGBM model for the current fold
#     lgb_model = lgb.LGBMRegressor(**lgb_params2)
#     lgb_model.fit(
#         df_fold_train[train_df2.columns],
#         df_fold_train_target,
#         eval_set=[(df_fold_valid[train_df2.columns], 
#                    df_fold_valid_target)],
#         callbacks=[
#             lgb.callback.early_stopping(stopping_rounds=100),
#             lgb.callback.log_evaluation(period=100),
#         ],
#     )

#     # Append the model to the list
#     models.append(lgb_model)
#     # Save the model to a file
# #     model_filename = os.path.join(model_save_path, f'doblez_{i+1}.txt')
# #     lgb_model.booster_.save_model(model_filename)
# #     print(f"Model for fold {i+1} saved to {model_filename}")

#     # Evaluate model performance on the validation set
#     fold_predictions = lgb_model.predict(df_fold_valid[feature_name])
#     fold_score = mean_absolute_error(fold_predictions, df_fold_valid_target)
#     scores.append(fold_score)
#     print(f"Fold {i+1} MAE: {fold_score}")

#     # Free up memory by deleting fold specific variables
#     del df_fold_train, df_fold_train_target, df_fold_valid, df_fold_valid_target
#     gc.collect()

In [ ]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

lgb_model.fit(train_df2, y)

In [ ]:
cnt = 0
for (test, revealed_targets, sample_predict) in iter_test:
    sample_predict["target"] = lgbmodel.predict(test.drop("row_id", axis=1))
    env.predict(sample_predict)
    cnt += 1